# Homework 4 Graded Exercise:
## Chapter7-9.ipynb (this file is provided only after grading) 
The  data  set  --    “Churn_Modelling.csv"  –  is  found  in  public  resource:  https://www.  kaggle. 
com/datasets/.  The  data  set  contains  a  bank’s  10,000  customers’  details,  viz.,  credit  score 
(higher  score  means  more  credit-worthy),  geography  (nationality),  gender,  age,  tenure  (years 
with the bank), bank balance in the period of  the data, the number of  bank products used by 
the customer, whether the customer has the bank credit card (yes = 1, no = 0), is active bank 
customer (yes =1, no = 0), and estimated USD salary of  the customer. The 10 features of  each 
customer  are  used  to  fit/predict  the  target  variable  “exited”  –  which  is  whether  the  customer 
exited  or  left  the  bank.  The  classification  of   “exited”  is  1  if   the  customer  left  the  bank. 
“exited” = 0 if  the customer continues with the bank.
The data follow that used in Chapter7-1 notebook. You should use the following as code line 
[1] in your solution. It sets a constant seed “5” for the random resampling for each epoch. This 
will ensure your computed numbers stay the same each time you re-run the program.


`In [1]: import json`\
` import numpy as np`\
` import tensorflow as tf`\
` import keras`\
` from keras import layers`\
` from keras import initializers`\
` keras.utils.set_random_seed(5)`\
 `tf.config.experimental.enable_op_determinism()`\
Next  you  should  use  the  same  code  lines  [2]  to  [8]  in  Chapter7-1.ipynb  to  feature-engineer  your 
data  such  as  hot-encoding,  including  preprocessing  via  standard-scaling,  and  split  your  data  into 
80% training set and 20% test set with random_state=1.
After this, deviate from the demonstration case in Chapter7-1. Employ tf.keras.models.Sequential() to 
construct a MultiLayer Perceptron with 3 hidden layers each with 6 neurons. Use sigmoid 
activation function. Compile using optimizer="adam",loss="binary_crossentropy",metrics=['accuracy’]. 
Use (mini) batch_size = 100 and number of  epochs = 500 to train/fit the MLP. 


In [1]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
from keras import initializers
keras.utils.set_random_seed(5)
tf.config.experimental.enable_op_determinism()


In [2]:
#Loading Dataset
data = pd.read_csv("QF634-G1-Applied Quantitative Research Methods - 11202024 - 802 PM/Churn_Modelling.csv")

In [3]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values
X = data.iloc[:,3:13]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
#Generating Dependent Variable Vectors
Y = data.iloc[:,-1].values
X = data.iloc[:,3:13]
X['Gender']=X['Gender'].map({'Female':0,'Male':1})
### above is used instead of a more complicated package involving -- from sklearn.preprocessing import LabelEncoder
### converts Female -- 0, Male -- 1, i.e. hot-encoding categorical variables
print (X['Gender'])

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64


In [5]:
#Encoding Categorical variable Geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct =ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
### Geography is transformed into France -- 1,0,0; Spain -- 0,0,1; Germany -- 0,1,0.
### Moreover -- this encoded vector of ones-zeros is now put in first 3 cols. Credit Score pushed to 4th col.

In [6]:
### convert X to dataframe X1
X1 = pd.DataFrame(X)
X1.head()
### Note there are 12 features including onehotencoder for the Geography feature-- 
### The features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,0.0,0.0,1.0,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,1.0,0.0,0.0,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,1.0,0.0,0.0,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,0.0,0.0,1.0,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10


In [7]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [8]:
#Performing Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
### This is the very first step while creating NNmodel. Here we are going to create our ann object by using a certain class of Keras 
### named Sequential. As a part of tensorflow 2.0, Keras is now integrated with tensorflow and is now considered as a 
### sub-library of tensorflow. The Sequential class is a part of the models module of Keras library which is a part of the 
### tensorflow library now. 
### It used to be "import tensorflow as tf; from tensorflow import keras; from tensorflow.keras import layers"
### See documentation at https://keras.io/guides/sequential_model/

#Initialising the NN model name -- NNmodel
NNmodel = tf.keras.models.Sequential()

In [10]:
### Creating a network that has 1 hidden layer together with 1 input layer and 1 output layer. 
#Adding First Hidden Layer
NNmodel.add(tf.keras.layers.Dense(units=6,activation="sigmoid"))
### units = 2 refer to 2 neurons in hidden layer 
### modelname.add is used to add a layer to the neural network -- need to specify as an argument what type of layer --
### Dense is used to specify the fully connected layer - i.e. all neurons are forward connect to all forward layer nodes

In [11]:
### Creating 2nd hidden layer 
#Adding Second Hidden Layer -- note this is added sequentially to the first hidden layer
NNmodel.add(tf.keras.layers.Dense(units=6,activation="sigmoid"))
### units = 8 refer to 8 neurons in hidden layer 

In [12]:
### Creating 3rd hidden layer 
#Adding Second Hidden Layer -- note this is added sequentially to the first hidden layer
NNmodel.add(tf.keras.layers.Dense(units=6,activation="sigmoid"))
### units = 8 refer to 8 neurons in hidden layer 

In [13]:
### now we create the output layer -- this is added sequentially
#Adding Output Layer
NNmodel.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
### Only 1 output neuron

In [14]:
### After creating the layers -- require compiling the NNmodel. Compiling allows the computer to run and understand the program 
### without the need of more fundamental steps in the programming. Compiling adds other elements or linking other libraries, and optimization,
### such that after compiling the results are readily computed e.g. in a binary executable program as an output. 
#Compiling NNmodel
NNmodel.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
### Note optimizer here is a more sophisticated version of the Mean Square loss

In [15]:
X_train.shape

(8000, 12)

In [16]:
#### Last step in creation of NNmodel. NNmodel is trained on the training set here with Tensor-Keras .fit based on Compiler
#Fitting NNmodel
history = NNmodel.fit(X_train,Y_train,batch_size=100,epochs = 500)
for i, acc in enumerate(history.history['accuracy']):
    if acc > 0.8:  # 0.8 corresponds to 80%
        print(f"Training accuracy exceeded 80% at epoch {i+1}")
        break

Epoch 1/500


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8003 - loss: 0.6082 
Epoch 2/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.8003 - loss: 0.5401
Epoch 3/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.8003 - loss: 0.5106
Epoch 4/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8003 - loss: 0.5008
Epoch 5/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.8003 - loss: 0.4975
Epoch 6/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.8003 - loss: 0.4954
Epoch 7/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.8003 - loss: 0.4933
Epoch 8/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.8003 - loss: 0.4908
Epoch 9/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - accuracy: 0.8003 - loss: 0.4875
Epoch 10/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.8003 - loss: 0.4833
Epoch 11/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.8003 - loss: 0.4779
Epoch 12/500
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - 

In [35]:
history.history['accuracy'][20:50]

[0.797249972820282,
 0.797249972820282,
 0.797249972820282,
 0.797249972820282,
 0.7977499961853027,
 0.8002499938011169,
 0.8090000152587891,
 0.8142499923706055,
 0.8190000057220459,
 0.8209999799728394,
 0.8221250176429749,
 0.8227499723434448,
 0.8234999775886536,
 0.824874997138977,
 0.8258749842643738,
 0.8268749713897705,
 0.828125,
 0.8286250233650208,
 0.8289999961853027,
 0.8291249871253967,
 0.8295000195503235,
 0.8297500014305115,
 0.8306249976158142,
 0.8308749794960022,
 0.831250011920929,
 0.8330000042915344,
 0.8341249823570251,
 0.8363749980926514,
 0.8388749957084656,
 0.840499997138977]

Q1. What is the total number of  parameters in this constructed MLP?

In [17]:
NNmodel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (100, 6)               │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (100, 6)               │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (100, 6)               │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (100, 1)               │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 509 (1.99 KB)

 Trainable params: 169 (676.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 340 (1.33 KB)

Q2.  What is the training accuracy after 500 epochs?

In [18]:
### Now we use the trained NNmodel to predict output in X_train sample
NNmodel.evaluate(X_train,Y_train)  ### evaluates the loss and accuracy as specified in the Compiler

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 723us/step - accuracy: 0.8687 - loss: 0.3266


[0.3259230852127075, 0.8648750185966492]

Q3.  What is the test accuracy (prediction accuracy on test sample) using the trained MLP?

In [19]:
### Now we use the trained NNmodel to predict output in X_train sample
NNmodel.evaluate(X_test,Y_test)  ### evaluates the loss and accuracy as specified in the Compiler

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.8659 - loss: 0.3265


2024-12-14 16:16:19.591483: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2024-12-14 16:16:19.592077: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

[0.3267632722854614, 0.8669999837875366]

Q4.  What is the number of  iterations before the training accuracy climbed above 80%?

Q5. A generalized data (not in the data set) of  a customer is a Spanish gentleman, 60 years old,
       with credit score 650, tenure 2 years with the bank, a balance of  USD 300,000, estimated
 salary of  USD 80,000, subscribing to two products, has a credit card, is assessed not to be an
       active member/customer. For ease of  interpretation, assume the exit prediction is for a year
       forward. Does this MLP model predict if  this customer will exit within the year?


In [20]:
data.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [21]:
x_test1 = pd.DataFrame([[650,"Spain", 1, 60, 2, 300_000.0, 2, 1, 0, 80_000.0]],
                       columns=data.columns[3:13])
x_test1 = ct.transform(x_test1)
x_test1

array([[0.0e+00, 0.0e+00, 1.0e+00, 6.5e+02, 1.0e+00, 6.0e+01, 2.0e+00,
        3.0e+05, 2.0e+00, 1.0e+00, 0.0e+00, 8.0e+04]])

In [22]:
x_test1 = sc.transform(x_test1)
x_test1

array([[-9.98501123e-01, -5.83123916e-01,  1.74601919e+00,
        -3.24425289e-03,  9.15090654e-01,  2.01683866e+00,
        -1.04828130e+00,  3.58411758e+00,  8.02256955e-01,
         6.43760170e-01, -1.02327069e+00, -3.49886744e-01]])

In [23]:
print(NNmodel.predict(x_test1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
[[0.746129]]
